<a href="https://colab.research.google.com/github/atlas-github/20190731SMG/blob/master/3B_Estimating_the_correlation_between_two_variables_with_a_contingency_table_and_a_chi_squared_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_This is one of the 100+ free recipes of the [IPython Cookbook, Second Edition](https://ipython-books.github.io/), by [Cyrille Rossant](http://cyrille.rossant.net/), a guide to numerical computing and data science in the Jupyter Notebook. The ebook and printed book are available for purchase at [Packt Publishing](https://www.packtpub.com/big-data-and-business-intelligence/ipython-interactive-computing-and-visualization-cookbook-second-e)._

+ ▶  _[Text on GitHub](https://github.com/ipython-books/cookbook-2nd) with a [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)_
+ ▶  _[Code on GitHub](https://github.com/ipython-books/cookbook-2nd-code) with a [MIT license](https://opensource.org/licenses/MIT)_
+ ▶  _[Go to Chapter 7 : Statistical Data Analysis](https://ipython-books.github.io/chapter-7-statistical-data-analysis/)_
+ ▶  _[Get the Jupyter notebook](https://github.com/ipython-books/cookbook-2nd-code)_

Whereas univariate methods deal with single-variable observations, multivariate methods consider observations with several features. Multivariate datasets allow the study of _relations_ between variables, more particularly their correlation, or lack thereof (that is, independence).

In this recipe, we will take a look at the same tennis dataset as in the first recipe of this chapter. Following a frequentist approach, we will estimate the correlation between the number of aces and the proportion of points won by a tennis player.

# How to do it...

1. Let's import NumPy, pandas, SciPy.stats, and matplotlib:

In [0]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
%matplotlib inline

2. We download and load the dataset:

In [0]:
player = 'Roger Federer'
df = pd.read_csv('https://github.com/ipython-books/'
                 'cookbook-2nd-data/blob/master/'
                 'federer.csv?raw=true',
                 parse_dates=['start date'],
                 dayfirst=True)

3. Each row corresponds to a match, and the 70 columns contain many player characteristics during that match:

In [0]:
print(f"Number of columns: {len(df.columns)}")
df[df.columns[:4]].tail()

4. Here, we only look at the proportion of points won, and the (relative) number of aces:

In [0]:
npoints = df['player1 total points total']
points = df['player1 total points won'] / npoints
aces = df['player1 aces'] / npoints
fig, ax = plt.subplots(1, 1)
ax.plot(points, aces, '.')
ax.set_xlabel('% of points won')
ax.set_ylabel('% of aces')
ax.set_xlim(0., 1.)
ax.set_ylim(0.)

If the two variables were independent, we would not see any trend in the cloud of points. On this plot, it is a bit hard to tell. Let's use pandas to compute a coefficient correlation.

5. For simplicity, we create a new __DataFrame__ object with only these fields. We also remove the rows where one field is missing (using __dropna()__):

In [0]:
df_bis = pd.DataFrame({'points': points,
                       'aces': aces}).dropna()
df_bis.tail()

6. Let's compute the Pearson's correlation coefficient between the relative number of aces in the match, and the number of points won:

In [0]:
df_bis.corr()

A correlation of ~0.26 seems to indicate a positive correlation between our two variables. In other words, the more aces in a match, the more points the player wins (which is not very surprising!).

7. Now, to determine if there is a statistically significant correlation between the variables, we use a __chi-squared test__ of the independence of variables in a __contingency table__. 
8.  First, we binarize our variables. Here, the value corresponding to the number of aces is __True__ if the player is serving more aces than usual in a match, and __False__ otherwise:

In [0]:
df_bis['result'] = (df_bis['points'] >
                    df_bis['points'].median())
df_bis['manyaces'] = (df_bis['aces'] >
                      df_bis['aces'].median())

9. Then, we create a contingency table, with the frequencies of all four possibilities (True and True, True and False, and so on):

In [0]:
pd.crosstab(df_bis['result'], df_bis['manyaces'])

10. Finally, we compute the chi-squared test statistic and the associated p-value. The null hypothesis is the independence between the variables. SciPy implements this test in __scipy.stats.chi2_contingency()__, which returns several objects. We're interested in the second result, which is the p-value:

In [0]:
st.chi2_contingency(_)

The p-value is much lower than 0.05, so we reject the null hypothesis and conclude that there is a statistically significant correlation between the proportion of aces and the proportion of points won in a match in this dataset.

Correlation does not imply causation. Here, it is likely that external factors influence both variables. See https://en.wikipedia.org/wiki/Correlation_does_not_imply_causation for more details.

# How it works...

We give here a few details about the statistical concepts used in this recipe.

## Pearson's correlation coefficient

Pearson's correlation coefficient measures the linear correlation between two random variables, $X$ and $Y$. It is a normalized version of the covariance:

$$ρ=\frac{cov(X,Y)}{\sqrt {var(X)var(Y)}} = \frac{E((X−E(X))(Y−E(Y)))}{\sqrt {var(X)var(Y)}}$$

It can be estimated by substituting, in this formula, the expectancy with the sample mean, and the variance with the sample variance. More details about its inference can be found at https://en.wikipedia.org/wiki/Pearson_product-moment_correlation_coefficient.

## Contingency table and chi-squared test 

The contingency table contains the frequencies $O_{ij}$ of all combinations of outcomes, when there are multiple random variables that can take a finite number of values. Under the null hypothesis of independence, we can compute the _expected frequencies_ $E_{ij}$, based on the marginal sums (sums in each row). The chi-squared statistic, by definition, is:

$$χ=∑_{i,j} \frac{(O_{ij}−E_{ij})^2}{E_{ij}}$$

When there are sufficiently many observations, this variable approximately follows a chi-squared distribution (the distribution of the sum of normal variables squared). Once we get the p-value, as explained in the _Getting started with statistical hypothesis testing – a simple z-test_ recipe, we can reject or accept the null hypothesis of independence. Then, we can conclude (or not) that there exists a significant correlation between the variables.

# There's more...

There are many other sorts of chi-squared tests, that is, tests where the test statistic follows a chi-squared distribution. These tests are widely used for testing the goodness-of-fit of a distribution, or testing the independence of variables. More information can be found in the following pages:

+ Chi2 test in SciPy documentation available at http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html
+ Contingency table introduced at https://en.wikipedia.org/wiki/Contingency_table
+ Chi-squared test introduced at https://en.wikipedia.org/wiki/Pearson's_chi-squared_test